In [15]:
import logging
logging.basicConfig(level=logging.INFO)

import numpy

import cicada.shamir
from cicada.communicator import SocketCommunicator

def main(communicator):
    log = cicada.Logger(logging.getLogger(), communicator)
    protocol = cicada.shamir.ShamirBasicProtocol(communicator, threshold=3)

    value = numpy.array(2.5) if communicator.rank == 0 else None
    log.info(f"Player {communicator.rank} value: {value}")
    
    value_share = protocol.share(secret=protocol.encoder.encode(value), src=0, shape=())
    double_share = protocol.add(value_share, value_share)
    double = protocol.encoder.decode(protocol.reveal(double_share))

    log.info(f"Player {communicator.rank} doubled value: {double}")

SocketCommunicator.run(world_size=3, fn=main);

INFO:root:Player 0 value: 2.5
INFO:root:Player 1 value: None
INFO:root:Player 2 value: None
INFO:root:Player 0 doubled value: 5.0
INFO:root:Player 1 doubled value: 5.0
INFO:root:Player 2 doubled value: 5.0


In [16]:
def main(communicator):
    log = cicada.Logger(logging.getLogger(), communicator)
    protocol = cicada.shamir.ShamirProtocol(communicator, threshold=3)

    value = numpy.array(2.5) if communicator.rank == 0 else None
    log.info(f"Player {communicator.rank} value: {value}")
    
    value_share = protocol.share(secret=protocol.encoder.encode(value), src=0, shape=())
    square_share = protocol.untruncated_multiply(value_share, value_share)
    square_share = protocol.truncate(square_share)
    square = protocol.encoder.decode(protocol.reveal(square_share))

    log.info(f"Player {communicator.rank} squared value: {square}")

SocketCommunicator.run(world_size=3, fn=main);

ERROR:cicada.communicator.socket:********************************************************************************
ERROR:cicada.communicator.socket:Comm world player 0 traceback:
ERROR:cicada.communicator.socket:Traceback (most recent call last):
  File "/Users/tshead/src/cicada-mpc/cicada/communicator/socket/__init__.py", line 812, in launch
    result = fn(communicator, *args, **kwargs)
  File "/var/folders/yw/yx17h14s5xs8kpsy1dtf8lhm0000gn/T/ipykernel_5343/1850389632.py", line 3, in main
    protocol = cicada.shamir.ShamirProtocol(communicator, threshold=3)
  File "/Users/tshead/src/cicada-mpc/cicada/shamir.py", line 557, in __init__
    raise ValueError(f'Threshold incompatible with worldsize, multiplications will not be feasible. Multiplications with this threshold would require worldsize at least {2*self.d+1}. Increase worldsize or decrease threshold.')
ValueError: Threshold incompatible with worldsize, multiplications will not be feasible. Multiplications with this threshold woul

In [17]:
def main(communicator):
    log = cicada.Logger(logging.getLogger(), communicator)
    protocol = cicada.shamir.ShamirProtocol(communicator, threshold=3)

    value = numpy.array(2.5) if communicator.rank == 0 else None
    log.info(f"Player {communicator.rank} value: {value}")
    
    value_share = protocol.share(secret=protocol.encoder.encode(value), src=0, shape=())
    square_share = protocol.untruncated_multiply(value_share, value_share)
    square_share = protocol.truncate(square_share)
    square = protocol.encoder.decode(protocol.reveal(square_share))

    log.info(f"Player {communicator.rank} squared value: {square}")

SocketCommunicator.run(world_size=5, fn=main);

INFO:root:Player 0 value: 2.5
INFO:root:Player 1 value: None
INFO:root:Player 2 value: None
INFO:root:Player 3 value: None
INFO:root:Player 4 value: None
INFO:root:Player 0 squared value: 6.25
INFO:root:Player 1 squared value: 6.25
INFO:root:Player 2 squared value: 6.25
INFO:root:Player 3 squared value: 6.25
INFO:root:Player 4 squared value: 6.25


Thus, Shamir offers straightforward resilience against data loss by allowing a subset of players to reveal data even after some players fail; however, this comes at a cost: it is less resistant to coalitions, since a sufficiently large coalition could reveal a secret that would otherwise remain private using additive sharing. Further, greater care is required to ensure that the number of players and threshold are chosen correctly, depending on the set of operations to be performed.